In [1]:
import os, sys
import xmltodict
import lxml
from astropy.io.votable import parse

base_dir = os.getcwd()
sys.path.append(os.path.join(base_dir, "python") )
data_dir = os.path.join(base_dir, "python", "tests", "data", "annotated_data") 

from utils.xml_utils import XmlUtils
from utils.dict_utils import DictUtils
from client.xml_interpreter.model_viewer import ModelViewer

print(sys.path)
print(base_dir)
print(data_dir)


   INFO - [__init__.py:  7 -   <module>()] - utils package initialized
   INFO - [__init__.py: 15 -   <module>()] - client package intialized
   INFO - [__init__.py: 14 -   <module>()] - translator package intialized
   INFO - [__init__.py: 14 -   <module>()] - translator package intialized
['/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/modelinstanceinvot-code', '/opt/miniconda3/envs/svom/lib/python38.zip', '/opt/miniconda3/envs/svom/lib/python3.8', '/opt/miniconda3/envs/svom/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/svom/lib/python3.8/site-packages', '/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/modelinstanceinvot-code/python']
/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/modelinstanceinvot-code
/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/modelinstanceinvot-code/python/tests/data/annotated_data


In [3]:
votable = parse(os.path.join(data_dir, "vizier_csc2_gal.annot.xml"))

In [5]:
for resource in votable.resources:
    mviewer = ModelViewer(resource, votable_path=os.path.join(data_dir, "vizier_csc2_gal.annot.xml"))
    break

   INFO - [model_viewer.py:267 - _extract_mapping_block()] - extract vodml block from /Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/modelinstanceinvot-code/python/tests/data/annotated_data/vizier_csc2_gal.annot.xml
   INFO - [annotation_seeker.py: 46 -   __init__()] - Found GLOBALS
   INFO - [annotation_seeker.py: 54 -   __init__()] - Found TEMPLATES Results
   INFO - [model_viewer.py:279 - _extract_mapping_block()] - VODML found


In [7]:
mviewer.connect_table('Results')


  DEBUG - [model_viewer.py:151 - connect_table()] - table Results found in VOTable
  DEBUG - [model_viewer.py:156 - connect_table()] - TEMPLATES Results found 


In [8]:
print(mviewer.get_next_row() )

   _RAJ2000      _DEJ2000        _2CXO          RAICRS      DEICRS     r0     r1       PA         GLON         GLAT      fe  fc  fp  fv fst  fs  fa  fi  fr  fm  ff fVi  Ns         PFluxb                 PFluxh                 PFluxm                 PFluxs                 PFluxu           PFluxw           PF90b                  PF90h                  PF90m                  PF90s                  PF90u            PF90w           Fluxb                Fluxh                Fluxm                Fluxs                Fluxu          Fluxw          F90b                 F90h                 F90m                 F90s                 F90u           F90w          PFavgb                 PFavgh                 PFavgm                 PFavgs                 PFavgu           PFavgw          PF90avgb               PF90avgh               PF90avgm               PF90avgs               PF90avgu         PF90avgw         Favgb                Favgh                Favgm                Favgs                Favgu  

In [10]:
XmlUtils.pretty_print(mviewer.get_model_view(resolve_ref=False))


<TEMPLATES tableref="Results">
			
				<INSTANCE dmrole="" dmtype="meas:Position">
					<ATTRIBUTE dmrole="meas:Position.ucd" dmtype="ivoa:string" value="pos"/>
					<INSTANCE dmrole="meas:Position.coord" dmtype="coord:LonLatPoint">
						<ATTRIBUTE dmrole="coord:LonLatPoint.lon" dmtype="ivoa:real" ref="RAICRS" index="3" value="253.923544"/>
						<ATTRIBUTE dmrole="coord:LonLatPoint.lat" dmtype="ivoa:real" ref="DEICRS" index="4" value="-42.8271581"/>
						<ATTRIBUTE dmrole="coord:LonLatPoint.dist" dmtype="ivoa:real" value=""/>
						<REFERENCE_1 dmrole="coords:Coordinate.coordSys" dmref="SpaceFrame_ICRS"/>
					</INSTANCE>
					<INSTANCE dmrole="meas:Measure.error" dmtype="meas:Ellipse">
						<COLLECTION dmrole="meas:Ellipse.semiAxis">
							<INSTANCE dmtype="ivoa:RealQuantity">
								<ATTRIBUTE dmrole="ivoa:Quantity.val" dmtype="ivoa:real" ref="r0" index="5" value="21.157"/>
								<ATTRIBUTE dmrole="ivoa:Quantity.unit" dmtype="ivoa:Unit" value="arcsec"/>
							</INSTANCE>
			

In [11]:
DictUtils.print_pretty_json(mviewer.get_json_model_view(resolve_ref=False))

  DEBUG - [model_viewer.py:228 - get_json_model_view()] - build json view
[
  [
    {
      "@dmrole": "",
      "@dmtype": "meas:Position",
      "meas:Position.ucd": "pos",
      "meas:Position.coord": {
        "@dmtype": "coord:LonLatPoint",
        "REFERENCE_1": {
          "@dmrole": "coords:Coordinate.coordSys",
          "@dmref": "SpaceFrame_ICRS"
        },
        "coord:LonLatPoint.lon": 253.923544,
        "coord:LonLatPoint.lat": -42.8271581,
        "coord:LonLatPoint.dist": null
      },
      "meas:Measure.error": {
        "@dmtype": "meas:Ellipse",
        "meas:Ellipse.semiAxis": [
          {
            "@dmtype": "ivoa:RealQuantity",
            "ivoa:Quantity.val": 21.157,
            "ivoa:Quantity.unit": "arcsec"
          },
          {
            "@dmtype": "ivoa:RealQuantity",
            "ivoa:Quantity.val": 13.738,
            "ivoa:Quantity.unit": "arcsec"
          }
        ],
        "meas:Ellipse.posAngle": {
          "@dmtype": "ivoa:RealQuantity

In [13]:
DictUtils.print_pretty_json(mviewer.get_json_model_component_by_type("coords:SpaceFrame"))

  DEBUG - [model_viewer.py:228 - get_json_model_view()] - build json view
[
  {
    "@dmid": "SpaceFrame_ICRS",
    "@dmtype": "coords:SpaceFrame",
    "coords:SpaceFrame.spaceRefFrame": "ICRS",
    "coords:SpaceFrame.equinox": "NoSet",
    "coords:SpaceFrame.refPosition": {
      "@dmtype": "coords:StdRefLocation",
      "coords:StdRefLocation.position": "NoSet"
    }
  },
  {
    "@dmid": "SpaceFrame_Galactic",
    "@dmtype": "coords:SpaceFrame",
    "coords:SpaceFrame.spaceRefFrame": "Galactic",
    "coords:SpaceFrame.equinox": "NoSet",
    "coords:SpaceFrame.refPosition": {
      "@dmtype": "coords:StdRefLocation",
      "coords:StdRefLocation.position": "NoSet"
    }
  }
]
